# Scraping User Activity

I'm trying to identify who the active users are on the market. Going to scrape the number of jobs that they have quouted in the last 30 days and use that as a measure of their activity

# Importing Libraries

In [1]:
# Packages for Scraping
from bs4 import BeautifulSoup
import requests
import random

# Packages for cleaning Data
import re
import pandas as pd
import numpy as np
import time

# Packages for PostgreSQL Import
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import os

# Other
import itertools

# Defining Functions

In [2]:
def html_extract():
    """
    Extracts and cleans the html from a website (x)
    
    Returns soup object from beautiful soup
    """
    source =  requests.get(html).text
    soup = BeautifulSoup(source, 'html.parser')
    
    return soup

In [3]:
def extract_30day_activity():
    """
    Visits a users profile and extracts their activity in the last 30 days.
    """
    activity_html = soup_html.find('ul',class_="dropdown-menu module_list tighter rhythmPadding1").find_all('li')
    activity_text = [x.get_text() for x in activity_html]
    activity_clean = [x.replace('\n','') for x in activity_text]
    activity_30days_text = activity_clean[2].split('\r')
    activity_30days_text = [x.strip() for x in activity_30days_text]
    activity_30days = int(activity_30days_text[1])
    
    return activity_30days

# Looping over all user htmls (I have scraped)

## Importing Data

In [4]:
# Names for accessing SQL database
dbname = "freelance_db"
username = os.environ['USER']
pswd = os.environ['SQLPSWD']

# Connect to SQL Database
con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

# Importing Data
sql_query = """SELECT profile_url FROM analysis_dummies_table;"""
dt = pd.read_sql_query(sql_query, con)

## Extracting activity in last 30 days

In [5]:
urls = list(dt.profile_url)

activity = []
for i, val in enumerate(urls):
    html = val
    
    print(i)
    
    soup_html = html_extract()
    try:
        activity.append([(html, extract_30day_activity())])
    except:
        activity.append([(html, "NA")])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

SSLError: HTTPSConnectionPool(host='www.guru.com', port=443): Max retries exceeded with url: /freelancers/kahshanna-evans (Caused by SSLError(SSLError("bad handshake: SysCallError(50, 'ENETDOWN')")))

In [ ]:
activity = [list(itertools.chain(*sub)) for sub in activity]
activity_df = pd.DataFrame(activity, columns = ['profile_url', 'quotes_30days'])

# Merging into Data

In [ ]:
# Importing Data
sql_query = """SELECT * FROM user_details_table;"""
user_details = pd.read_sql_query(sql_query, con)

# Prepping
user_details['profile_url'] = "https://www.guru.com"+user_details['profile_url']

# Merging
dt = pd.merge(user_details, activity_df, on='profile_url')
dt = dt.drop(['index'], axis = 1)

# Saving back out
engine = create_engine('postgresql://%s:%s@localhost/%s' %
                       (username, pswd, dbname))
dt.to_sql("user_details_table", engine, if_exists='replace')